In [2]:
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
import time

#Get all team data
nba_teams = pd.DataFrame(teams.get_teams())
#List of team ID's for commonroster api call
Team_ids = nba_teams.id.tolist()

#Function calls nba api for team roster info in a loop by team id
def teamroster_func(team_ids):
    
    #Lists to store roster and coach data for each team
    rosters = []
    coaches = []
    
    #for loop calls nba_api returns team rosters and coaches
    for id in team_ids:
        
        #Define Call
        call_commonteamroster = commonteamroster.CommonTeamRoster(team_id = id)
        
        #********team***************#
        #Get team roster data
        teamroster = call_commonteamroster.get_data_frames()[0]
        #build roster list
        rosters.append(teamroster)
        #turn list into dataframe
        team_rosters = pd.concat(rosters)
        
        #********coach*******************#
        #Get team coach data
        teamcoaches = call_commonteamroster.get_data_frames()[1]
        #build team coach list
        coaches.append(teamcoaches)
        #turn list into dataframe
        team_coaches = pd.concat(coaches)
        
        #******final_steps**********************#
        #Package roster and coach data
        teamData = team_rosters,team_coaches
        
        #Slows down the loop so api does not time out
        time.sleep(1)
    return(teamData)


#Call function
teamData = teamroster_func(Team_ids)

#Unpack team_rosters and team coaches dataframes
team_rosters, team_coaches = teamData


#******************Preprocess_data*********************************
#Get team id, abbreviation, and name 
team_names = nba_teams[['id','abbreviation','nickname']]
#rename columns for merge and to make easier to read
team_names = team_names.rename({'id':'TeamID','abbreviation':'Team_Abbreviation','nickname':'Team_Nickname'},axis='columns')
#rename team ID column for merge puposes
team_coaches = team_coaches.rename({'TEAM_ID':'TeamID'},axis='columns')

#Merge team name with roster and coaches dataframes
team_rosters = pd.merge(team_names,team_rosters)
team_coaches = pd.merge(team_names,team_coaches)

#Split height column into feet and inches columns
team_rosters[['HEIGHT_FEET','HEIGHT_INCHES']] = team_rosters['HEIGHT'].str.split('-',expand = True).astype('int64')
#Create new column total height in inches 
team_rosters['TOTAL_HEIGHT_INCHES'] = team_rosters.HEIGHT_FEET * 12 + team_rosters.HEIGHT_INCHES

#change TeamID and Player_ID to object data types
team_rosters[['TeamID','PLAYER_ID']] = team_rosters[['TeamID','PLAYER_ID']].astype('object')
#Replace R in EXP column with 0 Note: R = Rookie year
team_rosters['EXP'] = team_rosters.EXP.replace({'R':0})
#Change weight, age, exp into int64 data type
team_rosters[['WEIGHT','AGE','EXP']] = team_rosters[['WEIGHT','AGE','EXP']].astype('int64')



In [12]:
team_rosters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   TeamID               500 non-null    object
 1   Team_Abbreviation    500 non-null    object
 2   Team_Nickname        500 non-null    object
 3   SEASON               500 non-null    object
 4   LeagueID             500 non-null    object
 5   PLAYER               500 non-null    object
 6   NICKNAME             500 non-null    object
 7   PLAYER_SLUG          500 non-null    object
 8   NUM                  499 non-null    object
 9   POSITION             500 non-null    object
 10  HEIGHT               500 non-null    object
 11  WEIGHT               500 non-null    int64 
 12  BIRTH_DATE           500 non-null    object
 13  AGE                  500 non-null    int64 
 14  EXP                  500 non-null    int64 
 15  SCHOOL               500 non-null    object
 16  PLAYER_I

In [5]:
import altair as alt
from vega_datasets import data

source = data.population.url

alt.Chart(team_rosters).mark_boxplot(extent='min-max').encode(
    y = alt.Y('TOTAL_HEIGHT_INCHES',scale=alt.Scale(zero=False)),
    x = alt.X('Team_Abbreviation'))

/opt/anaconda3/envs/sports_analytics/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [40]:
team_averages = team_rosters.groupby(['Team_Abbreviation'],as_index=False)['EXP','AGE','TOTAL_HEIGHT_INCHES','WEIGHT'].mean()
alt.Chart(team_averages).mark_bar().encode(
    y = alt.Y('TOTAL_HEIGHT_INCHES'),
    x= alt.X('Team_Abbreviation',sort='y')
    )

/var/folders/71/8rtftxrj1njcvdvj90ttmthc0000gn/T/ipykernel_34806/761567316.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  team_averages = team_rosters.groupby(['Team_Abbreviation'],as_index=False)['EXP','AGE','TOTAL_HEIGHT_INCHES','WEIGHT'].mean()
/opt/anaconda3/envs/sports_analytics/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [36]:
team_averages.info()f



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Team_Abbreviation    30 non-null     object 
 1   EXP                  30 non-null     float64
 2   AGE                  30 non-null     float64
 3   TOTAL_HEIGHT_INCHES  30 non-null     float64
 4   WEIGHT               30 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.3+ KB
